# WordNet

In [1]:
import nltk

In [3]:
from nltk.corpus import wordnet as wn

In [8]:
terrier = wn.synsets('terrier')[0]

In [9]:
terrier.name

<bound method Synset.name of Synset('terrier.n.01')>

In [11]:
terrier.lemmas()

[Lemma('terrier.n.01.terrier')]

In [12]:
terrier.lemma_names()

['terrier']

In [17]:
dog = terrier.hypernyms()[0].hypernyms()[0]

In [58]:
def getHyponym(word): 
    return word.hyponyms()

def getHypernym(word): 
    return word.hypernyms()

In [25]:
dogTree = dog.tree(rel=getHyponym)

In [46]:
def isSynset(node): 
    return type(node) == nltk.corpus.reader.wordnet.Synset

In [44]:
isSynset(dogTree[0])

True

In [55]:
def getLemmas(tree): 
    allLemmas = []
    for node in tree: 
        if isSynset(node): 
            nodeLemmas = node.lemma_names()
            for lem in nodeLemmas: 
                allLemmas.append(lem)
        else: 
            nodeLemmas = getLemmas(node)
            for lem in nodeLemmas: 
                allLemmas.append(lem)
    return allLemmas

In [57]:
dogLemmas = getLemmas(dogTree)

In [59]:
dogHypers = dog.tree(getHypernym)

In [60]:
dogHypers

[Synset('dog.n.01'),
 [Synset('canine.n.02'),
  [Synset('carnivore.n.01'),
   [Synset('placental.n.01'),
    [Synset('mammal.n.01'),
     [Synset('vertebrate.n.01'),
      [Synset('chordate.n.01'),
       [Synset('animal.n.01'),
        [Synset('organism.n.01'),
         [Synset('living_thing.n.01'),
          [Synset('whole.n.02'),
           [Synset('object.n.01'),
            [Synset('physical_entity.n.01'),
             [Synset('entity.n.01')]]]]]]]]]]]]],
 [Synset('domestic_animal.n.01'),
  [Synset('animal.n.01'),
   [Synset('organism.n.01'),
    [Synset('living_thing.n.01'),
     [Synset('whole.n.02'),
      [Synset('object.n.01'),
       [Synset('physical_entity.n.01'), [Synset('entity.n.01')]]]]]]]]]

In [61]:
garden = open('../Readings/content/texts/garden-party.md').read()

In [68]:
gardenParts = garden.split('\n##')

In [71]:
garden.index('the weather was ideal')

74314

In [72]:
garden[74200:74400]

'as though it waked out of a\ndark dream. All was still.\n\n\n2. THE GARDEN PARTY.\n--------------------\n\nAnd after all the weather was ideal. They could not have had a more\nperfect day for a garden-party i'

In [73]:
garden.index('\n\n\n2. THE')

74254

In [74]:
garden.index('\n\n\n3. THE')

104246

In [76]:
gardenParty = garden[74254:104246]

In [114]:
gardenTokens = nltk.word_tokenize(gardenParty.lower())

In [82]:
# gardenAlphaTokens = [token for token in gardenTokens if token.isalpha()]

In [84]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.LancasterStemmer()

In [ ]:
lemmatizer.lemmatize

In [97]:
gardenLemmas = [lemmatizer.lemmatize(token) for token in gardenTokens]

In [90]:
dog.min_depth()

8

In [95]:
dog.hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0]

Synset('living_thing.n.01')

## POS Analysis: Parts of Speech

- Nouns: people, places, things
- Adjectives: descriptors: blue, happy, miserable, huge
- Verbs: actions: run, jump, write

POS Parsing / POS disambiguation

bank / bear

He deposited money at the bank. <- Bank (n) 
You can bank on it. <- Bank (v) 

He couldn't bear it. <- bear (v) 
He saw a terrifying bear! <- bear (n) 



In [115]:
gardenPOS = nltk.pos_tag(gardenTokens)

In [117]:
gardenNouns = set([word for word, pos in gardenPOS if pos == 'NN'])

- Walk up the hypernym tree for each noun, until the `min_depth()` is 3. 
- Categorize each noun according their hypernyms at that level. 

In [163]:
def getHypernymLevel3(word): 
    synsets = wn.synsets(word)
    if len(synsets) > 0: 
        synset = synsets[0]
        while synset.min_depth() > 4: 
            hypernyms = synset.hypernyms()
            if len(hypernyms) > 0:
                synset = hypernyms[0]
            else: 
                break
        return synset
    else: 
        return

## While Loops

```python
while (somecondition): 
    doSomething()
```

In [167]:
i = 0
while i < 10: 
    print(i)
    i += 1

0
1
2
3
4
5
6
7
8
9


In [135]:
getHypernymLevel3('plum')

Synset('living_thing.n.01')

In [139]:
livingThing = wn.synsets('living_thing')[0]

In [141]:
livingThing == getHypernymLevel3('plum')

True

In [147]:
getHypernymLevel3('oak_tree')

Synset('living_thing.n.01')

In [156]:
def isLivingThing(word): 
    hypernym = getHypernymLevel3(word)
    if hypernym is not None: 
        return getHypernymLevel3(word) == livingThing
    else: 
        return False

In [148]:
isLivingThing('oak_tree')

True

In [166]:
[token for token in list(gardenNouns) if isLivingThing(token)]

['egg',
 'bushes',
 'canna',
 'butterfly',
 'grass',
 'creature',
 'lavender',
 'horse',
 'dog',
 'sang',
 'mice',
 'daisy',
 'hen']

In [171]:
open('garden-party.txt', 'w').write(gardenParty)

29992

In [170]:
gardenParty

'\n\n\n2. THE GARDEN PARTY.\n--------------------\n\nAnd after all the weather was ideal. They could not have had a more\nperfect day for a garden-party if they had ordered it. Windless, warm,\nthe sky without a cloud. Only the blue was veiled with a haze of light\ngold, as it is sometimes in early summer. The gardener had been up since\ndawn, mowing the lawns and sweeping them, until the grass and the dark\nflat rosettes where the daisy plants had been seemed to shine. As for\nthe roses, you could not help feeling they understood that roses are the\nonly flowers that impress people at garden-parties; the only flowers\nthat everybody is certain of knowing. Hundreds, yes, literally hundreds,\nhad come out in a single night; the green bushes bowed down as though\nthey had been visited by archangels.\n\nBreakfast was not yet over before the men came to put up the marquee.\n\n"Where do you want the marquee put, mother?"\n\n"My dear child, it\'s no use asking me. I\'m determined to leave\ne